In [12]:
!pip install opencv-python pytesseract requests pandas spacy
!python -m spacy download en_core_web_sm


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.7 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [13]:
import cv2
import pytesseract
from google.colab.patches import cv2_imshow  # For displaying images in Colab

# Install and set up Tesseract (Google Colab)
!apt-get install -y tesseract-ocr
!pip install pytesseract
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

def extract_text_from_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    text = pytesseract.image_to_string(img)  # Extract text
    return text

# Upload the prescription image manually in Colab & set the path
image_path = "/content/prescription.jpg"

# Check if the image was loaded correctly
img = cv2.imread(image_path)
if img is None:
    print(f"Error: Could not load image from {image_path}. Please ensure the image file exists and is accessible.")
else:
    cv2_imshow(img)  # Show image in Colab
    prescription_text = extract_text_from_image(image_path)
    print("Extracted Text:\n", prescription_text)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Error: Could not load image from /content/prescription.jpg. Please ensure the image file exists and is accessible.


In [15]:
import cv2
import pytesseract
from google.colab.patches import cv2_imshow

# Install and setup Tesseract (Google Colab)
!apt-get install -y tesseract-ocr
!pip install pytesseract
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract"

def extract_text_from_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
    text = pytesseract.image_to_string(img)  # Extract text
    return text

# Upload prescription image manually in Colab & set the path
image_path = "/content/prescription.jpg"

# Check if the image was loaded correctly
img = cv2.imread(image_path)
if img is None:
    print(f"Error: Could not load image from {image_path}. Please ensure the image file exists and is accessible.")
else:
    cv2_imshow(img)  # Show image in Colab
    prescription_text = extract_text_from_image(image_path)  # ✅ Extract text
    print("Extracted Text:\n", prescription_text)

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Error: Could not load image from /content/prescription.jpg. Please ensure the image file exists and is accessible.


In [22]:
import spacy

# Load NLP model
nlp = spacy.load("en_core_web_sm")

def extract_medicines(text):
    if not text.strip():  # Check if text is empty
        return ["No text provided for extraction"]

    doc = nlp(text)
    medicines = [ent.text for ent in doc.ents if ent.label_ in ["ORG", "PRODUCT"]]

    return medicines if medicines else ["No medicines found"]

# Ensure `prescription_text` exists before running NLP
try:
    if 'prescription_text' in globals() and prescription_text.strip():
        medicines_list = extract_medicines(prescription_text)
        print("Extracted Medicines:", medicines_list)
    else:
        print("No text extracted from the image. Please check the OCR output.")
except NameError:
    print("Error: 'prescription_text' is not defined. Please extract text first.")


/usr/local/lib/python3.11/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


No text extracted from the image. Please check the OCR output.


In [23]:
import requests

# Replace with your actual API key
RAPIDAPI_KEY = "77a97fa331mshb5b64a725b65001p1ba2fbjsnc3f8537a919"
API_HOST = "superdrug-pricing-data.p.rapidapi.com"  # Check this on RapidAPI

def get_medicine_price(medicine_name):
    url = f"https://{API_HOST}/drug/search?query={medicine_name}"  # Correct endpoint
    headers = {
        "X-RapidAPI-Key": RAPIDAPI_KEY,
        "X-RapidAPI-Host": API_HOST
    }

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            data = response.json()
            if data:
                return data  # Return the extracted price data
            else:
                return "No price data available."
        else:
            return f"API Error: {response.status_code} - {response.text}"
    except Exception as e:
        return f"Request failed: {str(e)}"

# Ensure `medicines_list` exists before making API calls
if 'medicines_list' in globals() and medicines_list:
    for med in medicines_list:
        price_data = get_medicine_price(med)
        print(f"{med}: {price_data}")
else:
    print("No medicines found to fetch prices.")


No medicines found to fetch prices.
